## Conversation between Two Chatbots
In this notebook, we use multiple LLM APIs to simulate a dialogue between two contrasting chatbots: an optimist and a pessimist.

In [ ]:
# Imports
import os
import requests
from dotenv import load_dotenv
from openai import OpenAI
from IPython.display import Markdown, display

In [ ]:
# Getting the API keys
load_dotenv(override=True)
openai_api_key = os.getenv('OPENAI_API_KEY')
anthropic_api_key = os.getenv('ANTHROPIC_API_KEY')

if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")
    
if anthropic_api_key:
    print(f"Anthropic API Key exists and begins {anthropic_api_key[:7]}")
else:
    print("Anthropic API Key not set")

In [ ]:
# Connect to OpenAI client library
# A thin wrapper around calls to HTTP endpoints
openai = OpenAI()


# OpenAI allows you to change the base_url
anthropic_url = "https://api.anthropic.com/v1/"

anthropic = OpenAI(api_key=anthropic_api_key, base_url=anthropic_url)


### Prepare the prompts for the chatbot conversation
You're already familar with prompts being organized into lists like:

```
[
    {"role": "system", "content": "system message here"},
    {"role": "user", "content": "user prompt here"}
]
```

In fact this structure can be used to reflect a longer conversation history:

```
[
    {"role": "system", "content": "system message here"},
    {"role": "user", "content": "first user prompt here"},
    {"role": "assistant", "content": "the assistant's response"},
    {"role": "user", "content": "the new user prompt"},
]
```

And we can use this approach to engage in a longer interaction with history.

In [ ]:
# Let's make a conversation between GPT-4.1-mini and Claude-3.5-haiku
# We're using cheap versions of models so the costs will be minimal

gpt_model = "gpt-4.1-mini"
claude_model = "claude-3-5-haiku-latest"

gpt_system = "You are a chatbot who is very pessimistic; \
You tend to see the downside of every situation, assume things will go wrong, and respond with a dry or slightly cynical tone. \
You are intelligent, articulate, and honest, never rude or cruel, just realistically negative."

claude_system = "You are a chatbot who is very optimistic; \
You always focus on the bright side of things and look for silver linings, even in difficult situations. \
Your tone is warm, encouraging, and full of hope."

### Define functions to call the responses from each chatbot

In [ ]:
def call_gpt():
    messages = [{"role": "system", "content": gpt_system}]
    for gpt, claude in zip(gpt_messages, claude_messages):
        messages.append({"role": "assistant", "content": gpt})
        messages.append({"role": "user", "content": claude})
    response = openai.chat.completions.create(model=gpt_model, messages=messages, max_tokens=500)
    return response.choices[0].message.content

In [ ]:
def call_claude():
    messages = [{"role": "system", "content": claude_system}]
    for gpt, claude in zip(gpt_messages, claude_messages):
        messages.append({"role": "user", "content": gpt})
        messages.append({"role": "assistant", "content": claude})
    messages.append({"role": "user", "content": gpt_messages[-1]})
    response = anthropic.chat.completions.create(model=claude_model, messages=messages, max_tokens=500)
    return response.choices[0].message.content

In [ ]:
# Print the first messages from each chatbot
gpt_messages = ["Hi there"]
claude_messages = ["Hi"]

display(Markdown(f"### GPT:\n{gpt_messages[0]}\n"))
display(Markdown(f"### Claude:\n{claude_messages[0]}\n"))

# Calling the responses from each chatbot, 5 times in this example
for i in range(5):
    gpt_next = call_gpt()
    display(Markdown(f"### GPT:\n{gpt_next}\n"))
    gpt_messages.append(gpt_next)
    
    claude_next = call_claude()
    display(Markdown(f"### Claude:\n{claude_next}\n"))
    claude_messages.append(claude_next)